In [2]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from nltk.stem.snowball import SnowballStemmer

In [3]:
# load data
# reference: Final 02
test_data = pd.read_csv("/Users/yiwang/Documents/YiWang/Ebiz/Task 15/test-panda.csv",encoding ='latin1')
train_data = pd.read_csv('/Users/yiwang/Documents/YiWang/Ebiz/Task 15/train-panda.csv',encoding ='latin1')
attributes = pd.read_csv("/Users/yiwang/Documents/YiWang/Ebiz/Task 15/attributes-panda.csv",encoding ='latin1')
product_description = pd.read_csv("/Users/yiwang/Documents/YiWang/Ebiz/Task 15/product_descriptions-panda.csv",encoding ='latin1')

In [4]:
test_data.head()
train_data.head()
attributes.head()
product_description.head()

,product_uid,product_description
0,100001,"Not only do angles make joints stronger, they ..."
1,100002,BEHR Premium Textured DECKOVER is an innovativ...
2,100003,Classic architecture meets contemporary design...
3,100004,The Grape Solar 265-Watt Polycrystalline PV So...
4,100005,Update your bathroom with the Delta Vero Singl...


In [5]:
# merge attribute to find terms in query
# reference: https://www.kaggle.com/junfeng/home-depot-product-search-relevance/sklearn-random-forest-merge-attributes
def merge(attr):
    #names = attr["name"]
    values = attr["value"]
    merge = []
    for value in zip(values):
        merge.append(" ".join(value))
    return " ".join(merge)

In [6]:
attributes.dropna(how="all", inplace=True)
attributes["product_uid"] = attributes["product_uid"].astype(int)
attributes["value"] = attributes["value"].astype(str)
product_attributes = attributes.groupby("product_uid").apply(merge)
product_attributes = product_attributes.reset_index(name="product_attributes")

In [7]:
train_data = pd.merge(train_data, product_attributes, how="left", on="product_uid")

In [8]:
#train_data.drop("product_attributes_x")
#train_data.drop('product_description_x', axis=1, inplace=True)
#train_data.drop('product_attributes_x', axis=1, inplace=True)
train_data.head(10)

,id,product_uid,product_title,search_term,relevance,product_attributes
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,Versatile connector for various 90Â° connectio...
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,Versatile connector for various 90Â° connectio...
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,"Brush,Roller,Spray 6.63 in 7.76 in 6.63 in Rev..."
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,Combo Tub and Shower No Includes the trim kit ...
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,Combo Tub and Shower No Includes the trim kit ...
5,18,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,convection otr,3.00,Over the Range Microwave 18.5 in 17.13 in 29.9...
6,20,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,microwave over stove,2.67,Over the Range Microwave 18.5 in 17.13 in 29.9...
7,21,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,microwaves,3.00,Over the Range Microwave 18.5 in 17.13 in 29.9...
8,23,100007,Lithonia Lighting Quantum 2-Light Black LED Em...,emergency light,2.67,Ni-Cad .Built-In LED Advanced LED technology i...
9,27,100009,House of Fara 3/4 in. x 3 in. x 8 ft. MDF Flut...,mdf 3/4,3.00,Door & Window Made of primed MDF Can be painte...


In [11]:

train_data = pd.merge(train_data, product_description, how="left", on="product_uid")

In [13]:
#train_data.drop('product_description_x', axis=1, inplace=True)
train_data.drop('product_description_y', axis=1, inplace=True)
train_data.head(10)
#train_data.drop('product_attributes_y', axis=1, inplace=True)



,id,product_uid,product_title,search_term,relevance,product_attributes,product_description_x
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,Versatile connector for various 90Â° connectio...,"Not only do angles make joints stronger, they ..."
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,Versatile connector for various 90Â° connectio...,"Not only do angles make joints stronger, they ..."
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,"Brush,Roller,Spray 6.63 in 7.76 in 6.63 in Rev...",BEHR Premium Textured DECKOVER is an innovativ...
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,Combo Tub and Shower No Includes the trim kit ...,Update your bathroom with the Delta Vero Singl...
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,Combo Tub and Shower No Includes the trim kit ...,Update your bathroom with the Delta Vero Singl...
5,18,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,convection otr,3.00,Over the Range Microwave 18.5 in 17.13 in 29.9...,Achieving delicious results is almost effortle...
6,20,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,microwave over stove,2.67,Over the Range Microwave 18.5 in 17.13 in 29.9...,Achieving delicious results is almost effortle...
7,21,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,microwaves,3.00,Over the Range Microwave 18.5 in 17.13 in 29.9...,Achieving delicious results is almost effortle...
8,23,100007,Lithonia Lighting Quantum 2-Light Black LED Em...,emergency light,2.67,Ni-Cad .Built-In LED Advanced LED technology i...,The Quantum Adjustable 2-Light LED Black Emerg...
9,27,100009,House of Fara 3/4 in. x 3 in. x 8 ft. MDF Flut...,mdf 3/4,3.00,Door & Window Made of primed MDF Can be painte...,Get the House of Fara 3/4 in. x 3 in. x 8 ft. ...


In [14]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType,StringType,IntegerType

spark = SparkSession.builder.appName("test").getOrCreate()
sc = spark.sparkContext
sql_sc = SQLContext(sc)


sc = spark.sparkContext
sql_sc = SQLContext(sc)

trainSchema = StructType([
    StructField("id", IntegerType()),
    StructField("uid", IntegerType()),
    StructField("title", StringType()),
    StructField("term", StringType()),
    StructField("score", DoubleType()),
    StructField("attributes", StringType()),
    StructField("description", StringType())
])

rawTrain = sql_sc.createDataFrame(train_data, trainSchema)
rawTrain.show(5)

+---+------+--------------------+------------------+-----+--------------------+--------------------+
| id|   uid|               title|              term|score|          attributes|         description|
+---+------+--------------------+------------------+-----+--------------------+--------------------+
|  2|100001|Simpson Strong-Ti...|     angle bracket|  3.0|Versatile connect...|Not only do angle...|
|  3|100001|Simpson Strong-Ti...|         l bracket|  2.5|Versatile connect...|Not only do angle...|
|  9|100002|BEHR Premium Text...|         deck over|  3.0|Brush,Roller,Spra...|BEHR Premium Text...|
| 16|100005|Delta Vero 1-Hand...|  rain shower head| 2.33|Combo Tub and Sho...|Update your bathr...|
| 17|100005|Delta Vero 1-Hand...|shower only faucet| 2.67|Combo Tub and Sho...|Update your bathr...|
+---+------+--------------------+------------------+-----+--------------------+--------------------+
only showing top 5 rows



In [15]:
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import NGram
from pyspark.ml.feature import CountVectorizer

#title
    #tokenize
tokenizer = Tokenizer(inputCol="title", outputCol="title_words")
temp = tokenizer.transform(rawTrain)
    #idf
hashingTF = HashingTF(inputCol="title_words", outputCol="rawFeatures", numFeatures=10)
temp = hashingTF.transform(temp)

idf = IDF(inputCol = "rawFeatures", outputCol="title_idf")
idfModel = idf.fit(temp)
temp = idfModel.transform(temp)
temp=temp.drop("rawFeatures")
#     #n-gram , n = 2
# ngram = NGram(n=2, inputCol="title", outputCol="title_2grams")
# temp = ngram.transform(temp)
#     #n-gram , n = 3
# ngram = NGram(n=2, inputCol="title", outputCol="title_3grams")
# temp = ngram.transform(temp)


#temp=temp.drop('title')



#tokenize terms
tokenizer = Tokenizer(inputCol="term", outputCol="term_words")
temp = tokenizer.transform(temp)

hashingTF = HashingTF(inputCol="term_words", outputCol="rawFeatures", numFeatures=10)
temp = hashingTF.transform(temp)

idf = IDF(inputCol = "rawFeatures", outputCol="term_idf")
idfModel = idf.fit(temp)
temp = idfModel.transform(temp)
temp=temp.drop("rawFeatures")
#temp=temp.drop('term')

#tokenize attributes
tokenizer = Tokenizer(inputCol="attributes", outputCol="attr_words")
temp = tokenizer.transform(temp)

remover1 = StopWordsRemover(inputCol="attr_words", outputCol="attr_filtered")
temp = remover1.transform(temp)

hashingTF = HashingTF(inputCol="attr_filtered", outputCol="rawFeatures", numFeatures=10)
temp = hashingTF.transform(temp)

idf = IDF(inputCol = "rawFeatures", outputCol="attr_filtered_idf")
idfModel = idf.fit(temp)
temp = idfModel.transform(temp)
temp=temp.drop("rawFeatures")
#temp=temp.drop('attributes')


temp.show(3)
print(temp)

+---+------+--------------------+-------------+-----+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+
| id|   uid|               title|         term|score|          attributes|         description|         title_words|           title_idf|      term_words|            term_idf|          attr_words|       attr_filtered|   attr_filtered_idf|
+---+------+--------------------+-------------+-----+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+
|  2|100001|Simpson Strong-Ti...|angle bracket|  3.0|Versatile connect...|Not only do angle...|[simpson, strong-...|(10,[1,3],[0.5280...|[angle, bracket]|(10,[2,3],[1.2685...|[versatile, conne...|[versatile, conne...|(10,[0,1,2,3,4,5,...|
|  3|100001|Simpson Strong-Ti...|    l brack

In [16]:
#tokenize description
tokenizer = Tokenizer(inputCol="description", outputCol="descr_words")
temp = tokenizer.transform(temp)


remover2 = StopWordsRemover(inputCol="descr_words", outputCol="descr_filtered")
temp = remover2.transform(temp)

hashingTF = HashingTF(inputCol="descr_filtered", outputCol="rawFeatures", numFeatures=10)
temp = hashingTF.transform(temp)

idf = IDF(inputCol = "rawFeatures", outputCol="descr_filtered_idf")
idfModel = idf.fit(temp)
temp = idfModel.transform(temp)
temp=temp.drop("rawFeatures")
#temp=temp.drop("description")


#rawTrain.show(5)
temp.show(3)
print(temp)

+---+------+--------------------+-------------+-----+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| id|   uid|               title|         term|score|          attributes|         description|         title_words|           title_idf|      term_words|            term_idf|          attr_words|       attr_filtered|   attr_filtered_idf|         descr_words|      descr_filtered|  descr_filtered_idf|
+---+------+--------------------+-------------+-----+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  2|100001|Simpson Strong-Ti...|angle bracket|  3.0|Versatile connect...|Not only do angle...

In [55]:
import nltk

from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType,StringType,IntegerType
ps = PorterStemmer()

#get stem words for term, and description&title&attributes
def getStemWords(sentence):

    merge = []
    for s in sentence:
        merge = " ".join(ps.stem(s))
        return " ".join(merge)

stemUDF = udf(getStemWords, StringType())
test=temp.withColumn("title_stem", stemUDF("title_words"))
test.drop('attributes').drop('description').show(5) 

+---+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| id|   uid|score|         title_words|           title_idf|          term_words|            term_idf|          attr_words|            attr_idf|         descr_words|           descr_idf|       attr_filtered|      descr_filtered|          title_stem|
+---+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  2|100001|  3.0|[simpson, strong-...|(10,[1,3],[0.5280...|    [angle, bracket]|(10,[2,3],[1.2685...|[versatile, conne...|(10,[0,1,2,3,4,5,...|[not, only, do, a...|(10,[0,1,2,3,4,5,...|[versatile, conne...|[angles, make, jo...|s   i   m   p   s...|


In [17]:
print(temp)

DataFrame[id: int, uid: int, title: string, term: string, score: double, attributes: string, description: string, title_words: array<string>, title_idf: vector, term_words: array<string>, term_idf: vector, attr_words: array<string>, attr_filtered: array<string>, attr_filtered_idf: vector, descr_words: array<string>, descr_filtered: array<string>, descr_filtered_idf: vector]


In [18]:
from pyspark.sql.functions import udf
import math
def consineSimilarity(v1,v2):
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]
        y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return float(sumxy/math.sqrt(sumxx*sumyy))
idfUDF=udf(consineSimilarity,DoubleType())

result = temp
result = result.withColumn("term_title_idfCS", idfUDF("term_idf","title_idf"))
result = result.withColumn("term_attr_idfCS", idfUDF("term_idf","attr_filtered_idf"))
result = result.withColumn("term_descr_idfCS", idfUDF("term_idf","descr_filtered_idf"))
#result.show(5)
print(result)

DataFrame[id: int, uid: int, title: string, term: string, score: double, attributes: string, description: string, title_words: array<string>, title_idf: vector, term_words: array<string>, term_idf: vector, attr_words: array<string>, attr_filtered: array<string>, attr_filtered_idf: vector, descr_words: array<string>, descr_filtered: array<string>, descr_filtered_idf: vector, term_title_idfCS: double, term_attr_idfCS: double, term_descr_idfCS: double]


In [19]:
def matchWords(col,term):
    l1 = len(col)
    l2 = len(term)
    match = 0
    for i in range(l1):
        for j in range(l2):
            if col[i] == term[j]:
                match+=2
            elif col[i] in term[j]:
                match+=1
            elif term[j] in col[i]:
                match+=1
    return match
matchUDF=udf(matchWords, IntegerType())

result = result.withColumn("term_title_match", matchUDF("title_words","term_words"))
result = result.withColumn("term_attr_match", matchUDF("attr_filtered","term_words"))
result = result.withColumn("term_descr_match", matchUDF("descr_filtered","term_words"))
result.show(3)
print(result)

+---+------+--------------------+-------------+-----+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+-------------------+------------------+----------------+---------------+----------------+
| id|   uid|               title|         term|score|          attributes|         description|         title_words|           title_idf|      term_words|            term_idf|          attr_words|       attr_filtered|   attr_filtered_idf|         descr_words|      descr_filtered|  descr_filtered_idf|  term_title_idfCS|    term_attr_idfCS|  term_descr_idfCS|term_title_match|term_attr_match|term_descr_match|
+---+------+--------------------+-------------+-----+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+-----

In [20]:
def diceCoefficient(col, term):
    setA = set(col)
    setB = set(term)
    overlap = len(setA & setB)
    return float(overlap * 2.0 / (len(setA) + len(setB)))
diceUDF = udf(diceCoefficient, DoubleType())

result = result.withColumn("term_title_dice", diceUDF("title_words","term_words"))
result = result.withColumn("term_attr_dice", diceUDF("attr_filtered","term_words"))
result = result.withColumn("term_descr_dice", diceUDF("descr_filtered","term_words"))
#result.show(3)                                                      
print(result)

DataFrame[id: int, uid: int, title: string, term: string, score: double, attributes: string, description: string, title_words: array<string>, title_idf: vector, term_words: array<string>, term_idf: vector, attr_words: array<string>, attr_filtered: array<string>, attr_filtered_idf: vector, descr_words: array<string>, descr_filtered: array<string>, descr_filtered_idf: vector, term_title_idfCS: double, term_attr_idfCS: double, term_descr_idfCS: double, term_title_match: int, term_attr_match: int, term_descr_match: int, term_title_dice: double, term_attr_dice: double, term_descr_dice: double]


In [21]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer,StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark import SparkContext
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType,StringType,IntegerType
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler

features = ["term_title_match", "term_attr_match","term_descr_match","term_title_dice","term_attr_dice","term_descr_dice","term_title_idfCS", "term_attr_idfCS", "term_descr_idfCS"]

assember_features = VectorAssembler(inputCols=features, outputCol="features")
data = assember_features.transform(result)


In [22]:
data.select("features").show(3,truncate=False)

+-------------------------------------------------------------------------------------------------+
|features                                                                                         |
+-------------------------------------------------------------------------------------------------+
|[2.0,1.0,3.0,0.3333333333333333,0.0,0.0,0.6497180409001693,0.4825572980791222,0.3617569593147137]|
|[1.0,12.0,16.0,0.0,0.0,0.0,0.0,0.24405163592212437,0.3752210264145897]                           |
|[2.0,5.0,8.0,0.0,0.0,0.0,0.0,0.4124303071025185,0.3644175665256284]                              |
+-------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [23]:
data.registerTempTable("data")
trainData = sql_sc.sql("SELECT * from data where score is not NULL")
testData = sql_sc.sql("SELECT * from data where score is NULL")
trainData.select("id","uid","features").show(3,truncate=False)
testData.select("id","uid","features").show(3,truncate=False)

+---+------+-------------------------------------------------------------------------------------------------+
|id |uid   |features                                                                                         |
+---+------+-------------------------------------------------------------------------------------------------+
|2  |100001|[2.0,1.0,3.0,0.3333333333333333,0.0,0.0,0.6497180409001693,0.4825572980791222,0.3617569593147137]|
|3  |100001|[1.0,12.0,16.0,0.0,0.0,0.0,0.0,0.24405163592212437,0.3752210264145897]                           |
|9  |100002|[2.0,5.0,8.0,0.0,0.0,0.0,0.0,0.4124303071025185,0.3644175665256284]                              |
+---+------+-------------------------------------------------------------------------------------------------+
only showing top 3 rows

+---+---+--------+
|id |uid|features|
+---+---+--------+
+---+---+--------+



In [24]:
(trainD, validD) = trainData.randomSplit([0.8, 0.2])
#featureIndexer =VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=5).fit(trainD)
rf = RandomForestRegressor(featuresCol="features",labelCol='score', numTrees=11,maxDepth=5)
pipeline = Pipeline(stages=[rf])
model = pipeline.fit(trainD)
predictions = model.transform(validD)

In [25]:
evaluator = RegressionEvaluator(labelCol="score",predictionCol="prediction")
rmse = evaluator.evaluate(predictions,{evaluator.metricName:"rmse"})
mse = evaluator.evaluate(predictions,{evaluator.metricName:"mse"})
r2 = evaluator.evaluate(predictions,{evaluator.metricName:"r2"})
mae = evaluator.evaluate(predictions,{evaluator.metricName:"mae"})
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
print("Mean Squared Error (MSE) on test data = %g" % mse)
print("R^2 metric (R2) on test data = %g" % r2)
print("Mean Absolute Error (MAE) on test data = %g" % mae)

Root Mean Squared Error (RMSE) on test data = 0.506939
Mean Squared Error (MSE) on test data = 0.256987
R^2 metric (R2) on test data = 0.0843933
Mean Absolute Error (MAE) on test data = 0.415661
